In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()  # loads variables from .env
AZURE_ENDPOINT=os.getenv('AZURE_ENDPOINT')
AZURE_API_KEY=os.getenv('AZURE_API_KEY')

In [ ]:
# Initialize the same embeddings model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load the existing vector store
vectorstore = Chroma(
    persist_directory="../job_profiles_db",
    embedding_function=embeddings,
    collection_name="job_profiles"
)

In [4]:
# Retrieve context documents
query = "Which job profiles have software developer releated skills?"
context_docs = vectorstore.similarity_search(query, k=3)
context = "\n\n".join([doc.page_content for doc in context_docs])

In [5]:
print(context)

title: Dev Ops Specialist
overview: This position is responsible for creating and maintaining a fully automated DevOps development and production environment including automated code migration, and testing processes and tools for effective iterative software development. To create technical design specifications, evaluate existing applications and platforms, and review application modules for quality assurance. To evaluate new application packages and tools and develop user guides and training documentation.

title: Program Advisor
overview: The Program Advisor is accountable to attract and select the global skills and business talent British Columbia needs.  The Program Advisor may work in either the Skills Immigration or Entrepreneur Immigration Units of the PNP. The Skills Immigration Unit supports BC employers to recruit and retain qualified foreign workers to help meet their current and future skill needs, and whose activities contribute to the Province’s economic development obje

In [6]:
PROMPT_TEMPLATE = f"""System: You're an expert in answering questions about a library of job profiles. Use this context:
{context}

User: {query}
Assistant: """

In [8]:
# Initialize Azure client
client = ChatCompletionsClient(
    endpoint=AZURE_ENDPOINT,
    credential=AzureKeyCredential(AZURE_API_KEY),
    model="Mistral-small"
)

# Create and send request
response = client.complete(
    messages=[
        {"role": "user", "content": PROMPT_TEMPLATE}
    ],
    max_tokens=300,
    temperature=0.7,
)

In [12]:
# Process response
if response and response.choices:
    print("Response:", response.choices[0].message.content)
else:
    print("No response received")

Response: Based on the provided information, the Dev Ops Specialist job profile has software developer-related skills. This role involves creating and maintaining a fully automated DevOps development and production environment, including automated code migration and testing processes.
